<a href="https://colab.research.google.com/github/perico85/Alphazero/blob/master/funcional_2_predicciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Lista de URLs de los archivos CSV
urls = [
    "https://www.football-data.co.uk/mmz4281/2324/E0.csv",
    "https://www.football-data.co.uk/mmz4281/2223/E0.csv",
    "https://www.football-data.co.uk/mmz4281/2122/E0.csv",
    "https://www.football-data.co.uk/mmz4281/2021/E0.csv",
    "https://www.football-data.co.uk/mmz4281/1920/E0.csv",
    # Añade más URLs si es necesario
]

# Codificar las etiquetas de resultado FTR y HTR
label_encoder = LabelEncoder()

# Función para cargar los datos desde una URL y procesarlos
def cargar_datos_desde_urls(urls):
    dataframes = []
    for url in urls:
        data = pd.read_csv(url)
        data['FTR'] = label_encoder.fit_transform(data['FTR'])
        data['HTR'] = label_encoder.fit_transform(data['HTR'])
        dataframes.append(data)
    return pd.concat(dataframes)

# Función para calcular las estadísticas de un equipo basado en sus enfrentamientos previos
def calcular_estadisticas_equipo(matches, team, home=True):
    if home:
        matches = matches[matches['HomeTeam'] == team]
        estadisticas = matches[['FTHG', 'FTR', 'HTHG', 'HTR', 'HS', 'HST', 'HF', 'HC', 'HY', 'HR']].mean()
    else:
        matches = matches[matches['AwayTeam'] == team]
        estadisticas = matches[['FTAG', 'FTR', 'HTAG', 'HTR', 'AS', 'AST', 'AF', 'AC', 'AY', 'AR']].mean()
    return estadisticas.values

# Seleccionar las características relevantes para los modelos
features = ['Home_FTHG', 'Home_FTR', 'Home_HTHG', 'Home_HTR', 'Home_HS', 'Home_HST', 'Home_HF', 'Home_HC', 'Home_HY', 'Home_HR', 'Away_FTAG', 'Away_FTR', 'Away_HTAG', 'Away_HTR', 'Away_AS', 'Away_AST', 'Away_AF', 'Away_AC', 'Away_AY', 'Away_AR']

# Función para entrenar y evaluar los modelos
def train_and_evaluate_model(data):
    X_train, X_test, y_train, y_test = train_test_split(data[features], data['FTR'], test_size=0.2, random_state=42)

    # Crear y entrenar el modelo de regresión lineal
    regression_model = LinearRegression()
    regression_model.fit(X_train, y_train)

    # Crear y entrenar el modelo de red neuronal
    ann_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')
    ])
    ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    ann_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

    # Evaluar los modelos en el conjunto de prueba
    regression_accuracy = regression_model.score(X_test, y_test)
    ann_loss, ann_accuracy = ann_model.evaluate(X_test, y_test)

    return regression_accuracy, ann_loss, ann_accuracy

# Cargar los datos de todas las URLs
data = cargar_datos_desde_urls(urls)

# Crear nuevas columnas para las estadísticas del equipo local y visitante
data[['Home_FTHG', 'Home_FTR', 'Home_HTHG', 'Home_HTR', 'Home_HS', 'Home_HST', 'Home_HF', 'Home_HC', 'Home_HY', 'Home_HR']] = data.apply(lambda row: calcular_estadisticas_equipo(data, row['HomeTeam'], home=True), axis=1, result_type='expand')
data[['Away_FTAG', 'Away_FTR', 'Away_HTAG', 'Away_HTR', 'Away_AS', 'Away_AST', 'Away_AF', 'Away_AC', 'Away_AY', 'Away_AR']] = data.apply(lambda row: calcular_estadisticas_equipo(data, row['AwayTeam'], home=False), axis=1, result_type='expand')

# Entrenar y evaluar los modelos con los datos combinados
regression_accuracy, ann_loss, ann_accuracy = train_and_evaluate_model(data)
print(f'Accuracy of Linear Regression: {regression_accuracy}')
print(f'Loss of ANN: {ann_loss}, Accuracy of ANN: {ann_accuracy}')

# Definir modelos globalmente
regression_model = LinearRegression()
regression_model.fit(data[features], data['FTR'])

ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(len(features),)),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])
ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ann_model.fit(data[features], data['FTR'], epochs=50, batch_size=32, validation_split=0.2)


print(f'Accuracy of Linear Regression: {regression_accuracy}')
print(f'Loss of ANN: {ann_loss}, Accuracy of ANN: {ann_accuracy}')
# Función para predecir el resultado del partido utilizando los dos modelos
def predict_result(home_stats, away_stats):
    match_stats = np.concatenate((home_stats, away_stats))  # Concatenar las estadísticas de los equipos local y visitante
    match_stats = pd.DataFrame(match_stats.reshape(1, -1), columns=features)  # Creación del DataFrame con los nombres
    regression_result = regression_model.predict(match_stats)[0]
    ann_result = ann_model.predict([match_stats])[0]

    # Decode label for regression result
    regression_label = label_encoder.inverse_transform([round(regression_result)])[0]

    # Decode label for ANN result
    ann_label = label_encoder.inverse_transform([ann_result.argmax()])[0]

    return regression_label, ann_label

# Ejemplo de uso
home_team = "Liverpool"  # Reemplazar con el nombre real del equipo local
away_team = "Arsenal"  # Reemplazar con el nombre real del equipo visitante

home_stats = calcular_estadisticas_equipo(data, home_team, home=True)
away_stats = calcular_estadisticas_equipo(data, away_team, home=False)


regression_result, ann_result = predict_result(home_stats, away_stats)
print(f'Resultado predicho por regresión lineal: {regression_result}')
print(f'Resultado predicho por red neuronal: {ann_result}')

Epoch 1/50
36/36 [==============================] - 1s 16ms/step - loss: 1.1109 - accuracy: 0.4816 - val_loss: 1.0254 - val_accuracy: 0.5000
Epoch 2/50
36/36 [==============================] - 0s 5ms/step - loss: 0.9990 - accuracy: 0.5210 - val_loss: 1.0091 - val_accuracy: 0.5105
Epoch 3/50
36/36 [==============================] - 0s 6ms/step - loss: 1.0012 - accuracy: 0.5122 - val_loss: 1.0088 - val_accuracy: 0.5315
Epoch 4/50
36/36 [==============================] - 0s 7ms/step - loss: 1.0025 - accuracy: 0.5253 - val_loss: 1.0083 - val_accuracy: 0.5210
Epoch 5/50
36/36 [==============================] - 0s 6ms/step - loss: 0.9939 - accuracy: 0.5184 - val_loss: 0.9866 - val_accuracy: 0.5245
Epoch 6/50
36/36 [==============================] - 0s 6ms/step - loss: 0.9898 - accuracy: 0.5253 - val_loss: 1.0068 - val_accuracy: 0.5245
Epoch 7/50
36/36 [==============================] - 0s 7ms/step - loss: 0.9866 - accuracy: 0.5245 - val_loss: 1.1308 - val_accuracy: 0.4161
Epoch 8/50
36/36 [=